<a href="https://colab.research.google.com/github/stat-junda/Stat-359-Modern-Deep-Learning/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Answering Fermi Questions

This assignment consists of three parts.

1. Basic usage of OpenAI's API and how to construct query calls
2. Writing good prompts to get better output from ChatGPT
3. Basics of the attention mechanism

In [13]:
# Uncomment the ! pip line of this cell if you get errors running cell 3.
# You might get errors when running this. If you are able to run cell 3, then
# you can ignore these errors.
! pip install cohere tiktoken
! pip install openai



# Part 1) OpenAI API

Read through the documentation of OpenAI API. Start [here](https://github.com/openai/openai-python).


You should have an API key. Place your API key in the left tool bar on Colab, by clicking on the tool item the shape of a key, labelled "secrets".

Add a new secret.

Give it the name "**openai**" and paste your key in the value section. Make sure to toggle Notebook access.



In [14]:
from typing import Dict
from google.colab import userdata
API_KEY = userdata.get('openai')

In [15]:
from openai import OpenAI, ChatCompletion

# Instantiate your OpenAI API client

client = OpenAI(
    api_key=API_KEY,
)

In [16]:
# The key has been set. We are ready to query ChatGPT.
# Fill out the functions below.
# Do not batch queries.
# Do not change the model from 'gpt-3.5-turbo'

def basic_query(client: OpenAI, message: str, model: str = 'gpt-3.5-turbo'):
  '''
  Inputs:
  `client`: Takes an OpenAI client object
  `message`: The message to pass to the OpenAI LLM model
  `model`: The model to use. Do not modify this

  Returns:
  An OpenAI response object to the message you have passed.
  '''
  messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": message}
    ] # fill out the message payload

  response = client.chat.completions.create(
        model=model,
        messages=messages
    ) # placeholder

  return response


In [17]:
def parse_response(responseObj: ChatCompletion) -> Dict[str, str]:
    '''
    Inputs:
    `responseObj`: The response of the GPT chat.completions.create()

    Returns:
    A dictionary of the following format:
    {
      'role': STRING representing the role attribute returned by the response obj
      'content': STRING representation of GPT's response message,
    }

    Hint, run the basic_query() in another code block and observe the output structure.
    That alone should be sufficient to make this function.
    '''
    # Extracting the last message from the response, which is the model's reply
    last_message = responseObj.choices[0].message

    # Constructing the output dictionary
    out = {
        'content': last_message.content
    }

    return out


In [18]:
# RUN BUT DO NOT MODIFY THE CODE BELOW

message = '''What is a Fermi Question.
Explain it concisely and provide a brief example.'''

responseObj = basic_query(client=client,
                          message= message)

parse_response(responseObj)

{'content': 'A Fermi question is a problem-solving technique that involves making rough estimations and using logical reasoning to arrive at an approximate answer. It is named after the physicist Enrico Fermi, who was known for his ability to make quick and accurate estimations.\n\nAn example of a Fermi question would be "How many piano tuners are there in the city of New York?" To tackle this question, you would break it down into smaller, more manageable components. For instance, you might estimate how many households in New York own pianos, how often they need tuning, and how many pianos a tuner can tune in a day. Then, by multiplying these approximations together, you can arrive at a reasonable estimate for the total number of piano tuners in New York.'}


# Part 2) Fermi Questions

In [19]:
question = '''
How many tennis balls can fit on a Boeing 747?
'''

fermiResponseObj = basic_query(client, message=question)
parse_response(fermiResponseObj)

{'content': 'The exact number of tennis balls that can fit on a Boeing 747 would depend on factors such as the model of the plane, the available space, and how the balls are packed. Without specific measurements and configurations, it is difficult to give an accurate answer. However, it is safe to say that a Boeing 747 has a large cargo hold and can accommodate thousands of tennis balls.'}

When I ran the code, I got the following result:

```
The exact number of tennis balls that can fit on a Boeing 747 would depend on various factors, such as the size of the tennis balls, how they are packed, and whether any other cargo is being carried in the plane.
Furthermore, there are different models of the Boeing 747 with varying cargo capacities.
However, as a rough estimate, to visualize the capacity, let's assume we are using standard-sized tennis balls (approximately 6.7 cm in diameter)
and considering only the cargo hold of a Boeing 747-400.

The cargo hold of a Boeing 747-400 has a total volume of around 570 cubic meters. Assuming that each tennis ball has a volume of approximately 26.8 cubic centimeters, we can calculate an approximate number.

570 cubic meters is equivalent to 570,000,000 cubic centimeters.

Thus, the estimated number of tennis balls that can fit would be:

570,000,000 cubic centimeters / 26.8 cubic centimeters ≈ 21,268,656 tennis balls.

Please note that this is a simplified calculation and does not take into account any additional factors that may affect the total capacity of the aircraft or its weight limitations.
```

This answer is not good for a couple of reasons
  - This seems like quite a relatively specific number rather than a range.
  - The more acceptable approach is to compare volumes. This compares weights. This is actually telling us how many tennis balls will it take to weight the same as a 747.

Your job is to make these estimates better with prompting. You should aim for a response that provides a reasonable range.

E.g. How many tennis balls fit in a Boeing 747?

Possible types of responses:

 - ~Between 100 and 200~. While true and obvious, this is not a very intelligent answer
 - ~60,000,000 tennis balls~. This answer provides no chain of thought.
 - Provided the diameter of a tennis ball is x, and the volume of the 747 is z, we estimate around 10^4 to 10^6 tennis balls can fit on a 747. (If your GPT gives this type of response, it would be considered an excellent response.)


Your job is to come up with one single prompt, that will answer all fifty Fermi questions below. In reality, you will probably encounter some questions that it cannot solve no matter how you prompt it. That is OK. Try to create a prompt for GPT to yield as many reasonable results as you can. Be creative with your  prompt.


**When testing out different prompts, only run it on 1 or 2 questions. Once you have decided on a prompt, go ahead and run it on the entire list.**


In [20]:
PROMPT = '''
When answering Fermi questions, such as estimating quantities or numbers in various scenarios, it's important to focus on logical reasoning and approximation.
Use known averages or typical values for measurements that are not specified.
Explain your reasoning in steps, highlighting any assumptions you make.
Always aim to provide answers in the form of a reasonable range to reflect the uncertainties involved in such estimations.
Your answer should give a general idea rather than a precise figure, demonstrating the thought process behind arriving at that estimate.
'''

In [21]:
# DO NOT MODIFY THESE QUESTIONS
questions = [
  'How many cups of coffee are consumed globally in a day?',
  'Estimate the number of bricks used in the construction of the Great Wall of China.',
  'How many grains of sand are on an average sized beach?',
  'Estimate the total weight of all elephants in Africa in kilograms.',
  'How many breaths does an average person take in a year?',
  'Estimate the number of hours the average person spends watching TV in a month.',
  'How many blades of grass are on a soccer field?',
  'Estimate the number of cells in the human brain.',
  'How many total hours of sleep does a person get in a year?',
  'Estimate the number of text messages sent globally in a minute.',
  'How many McDonalds are there in the United States?',
  'Estimate the total weight of all the plastic produced in a year.',
  'How many footsteps does an average person take in a day?',
  'Estimate the daily number of people who visit a large shopping mall.',
  'How many gigabytes of data are transferred on the internet in an hour?',
  'Estimate the number of busses in the United States.',
  'How many trumpet players are there in the world?',
  'Estimate the total length of all the rivers in Asia in kilometers.',
  'How many calories does the average person consume on Thanksgiving Day?',
  'Estimate the number of bacteria on an average human hand.',
  'How many books are in all libraries in the United States',
  'Estimate the number of atoms in a grain of sand.',
  'How many cups of tea are consumed daily around the world?',
  'Estimate the total weight of all the paper produced in a year.',
  'How many blades of wheat are in a small loaf of bread?',
  'Estimate the number of hairs on an average cat.',
  'How many breaths does a newborn baby take in an hour?',
  'Estimate the number of footsteps taken during a marathon.',
  'How many ants are there in a large ant colony?',
  'Estimate the number of stars visible in the night sky from a city.',
  'How many tweets are sent worldwide in a minute?',
  'Estimate the total weight of all the apples sold in a month.',
  'How many atoms are there in a human cell?',
  'Estimate the number of raindrops in a light rain shower.',
  'How many total minutes of music are streamed on Spotify in a day?',
  'Estimate the number of computer mice in use globally.',
  'How many light bulbs are in Times Square during New Years Eve?',
  'Estimate the total length of all the fiber optic cables under the ocean.',
  'How many wheels are there in the world?',
  'Estimate the number of grains of rice in a 5 kilogram bag.',
  'How many blades of grass are in an 18 hole golf course?',
  'Estimate the number of words in an average length novel.',
  'How many breaths does a person take during a one hour yoga session?',
  'Estimate the total weight of all the plastic bottles used in a year.',
  'How many footsteps does the average person take in a lifetime?',
  'Estimate the number of atoms in a cup of water.',
  'How many grains of salt are there in a typical salt shaker?',
  'Estimate the total length of all the subway tracks in New York City.',
  'How many iPhones are there in the world?',
  'Estimate the number of words spoken in the United States in one day.'
]

In [22]:
def answer_all(PROMPT: str, questions: list) -> list:
  '''
  Inputs:
  `PROMPT`: your prompt as a string
  `questions`: the list of questions from above

  Returns:
  A list of dictionary responses containing 'role' and 'content' keys for each question-response.


  Write a function to take in your prompt, append it to all the questions
  to send to the API with basic query. Do not use batching.

  Return your responses as dictionaries like above
  into a list called `answers`.

  You may use the functions you have written above:
  basic_query() and parse_response()

  '''

  ## WRITE YOUR FUNCTION HERE

  answers = [] # this is a placeholder

  # for testing purposes, you can evaluate your prompts on a single question.
  # once you are happy with your prompt, you can remove the [:1] below, to
  # loop over the entire question list.
  for question in questions:
        # Combine the prompt with the question
        combined_query = PROMPT + '\n\n' + question

        # Send the combined query to the basic_query function
        responseObj = basic_query(client, message=combined_query)

        # Parse the response and add it to the answers list
        parsed_response = parse_response(responseObj)
        answers.append(parsed_response)


  return answers


In [23]:
# RUN BUT DO NOT MODIFY THIS CODE BLOCK.
# This code block may take a while to run

answer_all(PROMPT=PROMPT, questions=questions)

[{'content': "To estimate the number of cups of coffee consumed globally in a day, we can start by breaking down the problem into smaller components:\n\n1. Determine the global population: As of 2021, the global population is approximately 7.9 billion people.\n\n2. Estimate the percentage of coffee drinkers: It is difficult to determine the exact percentage, but various sources suggest that around 40-45% of the global population consumes coffee regularly. For the sake of estimation, let's assume the percentage of coffee drinkers is 40%.\n\n3. Average cups of coffee consumed per day: Now, we need to estimate how many cups of coffee an average coffee drinker consumes in a day. This can vary greatly, but for simplicity, let's assume an average of 2 cups per day.\n\nWith these assumptions, we can calculate the estimate:\n\nGlobal population: 7.9 billion\nPercentage of coffee drinkers: 40%\nAverage cups per day: 2\n\nEstimated cups of coffee consumed globally per day:\n7.9 billion * 0.40 * 

# Part 3) Implementing Basic Attention

Finish this implementation of the attention mechanism. Your result should be an attention matrix with 4 rows and 3 columns.

Your job is to implement the `get_attention()` function

In [24]:
import numpy as np
from numpy import random
from numpy import dot
from scipy.special import softmax


class BasicAttention:
  def __init__(self):
    # Assume these word embeddings
    self.word_1 = np.array([1, 0, 0])
    self.word_2 = np.array([0, 1, 0])
    self.word_3 = np.array([1, 1, 0])
    self.word_4 = np.array([0, 0, 1])

    # Place the words into an array
    self.words = np.array([self.word_1, self.word_2, self.word_3, self.word_4])

    # Simulate the weight matrices
    # In an actual model, these matrices are learned via optimization
    rng = random.RandomState(seed=42)
    self.W_Q = rng.randint(3, size=(3, 3))
    self.W_K = rng.randint(3, size=(3, 3))
    self.W_V = rng.randint(3, size=(3, 3))

  def get_attention(self) -> np.ndarray:
    # Step 1: Calculate Q, K, V matrices
    Q = dot(self.words, self.W_Q)
    K = dot(self.words, self.W_K)
    V = dot(self.words, self.W_V)

    # Step 2: Compute the dot product of Q and the transpose of K,
    # then apply softmax to get attention scores
    attention_scores = softmax(dot(Q, K.T), axis=1)

    # Step 3: Multiply the attention scores with V to get the final attention output
    attn = dot(attention_scores, V)

    return attn


In [25]:
attn = BasicAttention()
print(attn.get_attention())

[[0.9994094  1.87998158 0.88057218]
 [0.98201379 1.48201379 0.5       ]
 [0.99998918 1.88078213 0.88079296]
 [0.99993902 1.98193751 0.98199849]]


## NOTE

### Due to the generative nature of ChatGPT, your answers can be stochastic when run at different times and may differ depending on your inputs. It is important that you run the code for you to see your response.

### Before submitting this assignment, RUN and KEEP the output results on this notebook. If you do not produce or keep these outputs, then the code will be run when it is being graded. Then you will be graded on the outputs from that run.

### Best to run your code beforehand, so you know what you are submitting.

